In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import glob
import gc

print(datetime.datetime.now())


def recursive_file_gen(root_folder):
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            yield os.path.join(root, file)
            
os.getcwd()

track_start_date="2018-09-01"

2019-11-18 19:02:09.876220


In [2]:
# July 1 2018 to Oct 2019

list_weekly_signs_files=list(recursive_file_gen("/home/jian/BigLots/"))
list_weekly_signs_files=[x for x in list_weekly_signs_files if "master" in x.lower()]
list_weekly_signs_files=[x for x in list_weekly_signs_files if x[-4:]==".txt"]
list_weekly_signs_files=[x for x in list_weekly_signs_files if "MediaStorm_" in x]
list_weekly_signs_files=[x for x in list_weekly_signs_files if "weekly" in x.lower()]

list_weekly_signs_files.sort()

In [3]:
df_new_signs_since_Sep18=pd.DataFrame()

for file in list_weekly_signs_files:
    df=pd.read_table(file,dtype=str,sep="|",usecols=['customer_id_hashed','email_address_hash','sign_up_date'])
    df=df[df['sign_up_date']>=track_start_date]
    df_new_signs_since_Sep18=df_new_signs_since_Sep18.append(df)

In [4]:
dfdetail_1=pd.read_table("/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/All Rewards Members 2018-02-04 - 2019-05-04.zip",
                         dtype=str,sep="|",usecols=['customer_id_hashed','email_address_hash','sign_up_date'])
dfdetail_2=pd.read_table("/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/MediaStorm Rewards Master P4 2019 - no transaction info.zip",
                         dtype=str,sep="|",usecols=['customer_id_hashed','email_address_hash','sign_up_date'])

dfdetail_1=dfdetail_1[dfdetail_1['sign_up_date']>=track_start_date]
dfdetail_2=dfdetail_2[dfdetail_2['sign_up_date']>=track_start_date]

df_new_signs_since_Sep18=df_new_signs_since_Sep18.append(dfdetail_1).append(dfdetail_2)

del dfdetail_1
del dfdetail_2

gc.collect()

0

In [5]:
print(df_new_signs_since_Sep18.shape)
df_new_signs_since_Sep18=df_new_signs_since_Sep18.sort_values("sign_up_date",ascending=False).drop_duplicates("customer_id_hashed")
print(df_new_signs_since_Sep18.shape)

(11600797, 3)
(8221880, 3)


In [6]:
month_date_count=df_new_signs_since_Sep18[['sign_up_date']].drop_duplicates()
month_date_count['registerd_month']=month_date_count['sign_up_date'].apply(lambda x: x[:7])
month_date_count=month_date_count.groupby("registerd_month")['sign_up_date'].nunique().to_frame().reset_index()

In [7]:
df_output_long=df_new_signs_since_Sep18[['customer_id_hashed','sign_up_date']]
df_output_long['registerd_month']=df_output_long['sign_up_date'].apply(lambda x: x[:7])
del df_new_signs_since_Sep18

list_ids_new_signs_since_Sep18=df_output_long['customer_id_hashed'].tolist()
gc.collect()

print(df_output_long.shape)
df_output_long.head(3)

(8221880, 3)


,customer_id_hashed,sign_up_date,registerd_month
140532,b427a86927696538066305dd91ae960734488531b44d81...,2019-11-09,2019-11
50861,4f78a082263f3bf1e27759770ba7199774b71f3078046d...,2019-11-09,2019-11
50831,434ee5bad15c849d2a4ce0cc97d1eb943fd2ff50e0b8e8...,2019-11-09,2019-11


In [8]:
min_month_str=df_output_long['registerd_month'].min()
min_date_str=df_output_long['sign_up_date'].min()
print(min_date_str,min_month_str)

2018-09-01 2018-09


In [9]:
# Sales all since 2018-09-01

list_hist_item=glob.glob("/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/*.txt")
list_hist_item=[x for x in list_hist_item if x.split("MediaStormDailySalesHistory")[1][:8]>=track_start_date.replace("-","")]
list_hist_item.sort()
last_weekend_dt_historical=max([x.split("MediaStormDailySalesHistory")[1][:8] for x in list_hist_item])
print(last_weekend_dt_historical[:4]+"-"+last_weekend_dt_historical[4:6]+"-"+last_weekend_dt_historical[6:8])

2019-02-09


In [10]:
list_weekly_item=list(recursive_file_gen("/home/jian/BigLots/"))
list_weekly_item=[x for x in list_weekly_item if "daily" in x.lower() and x[-4:]==".txt"]
list_weekly_item=[x for x in list_weekly_item if "/MediaStorm_" in x]
list_weekly_item.sort()
list_weekly_item=[x for x in list_weekly_item if x.split("/MediaStorm_")[1][:10]>last_weekend_dt_historical[:4]+"-"+last_weekend_dt_historical[4:6]+"-"+last_weekend_dt_historical[6:8]]
# list_weekly_item


list_all_POS_item=list_hist_item+list_weekly_item
print(len(list_all_POS_item))

63


In [11]:
pd.read_table(file,dtype=str,sep="|",nrows=10)

,customer_id_hashed,email_address_hash,sign_up_date,sign_up_channel,sign_up_location,customer_zip_code,transaction_count,transaction_amount,experian_multi_cluster,experian_demo_cluster,purchase_channel,email_unsubscribe_indicator,email_undeliverable_indicator
0,ed80617a839f59cacf58caf76a8739a6594bc85952aff3...,734755e94333b15fe2f07c820d5583a62dcece657587de...,2019-11-07,STORE,4693,98682,000000000000001.,0000000000000051.48,0,0,STORES,N,N
1,2f94802cc5241ada7ad83422b04a3305508e95ff35d779...,04fd2034cd7639334a107a9c83b2d17048df35d12cccd5...,2019-11-03,ONLINE,6990,NaN,000000000000000.,0000000000000000.00,0,0,NONE,N,N
2,029091a00568ade1ab22809423124e4338ee6b64c3bbf4...,e60c4dc9a20624fa5b36d1a0264050da050e910dc3a443...,2019-11-04,STORE,5325,37048,000000000000001.,0000000000000499.99,0,0,STORES,N,N
3,5705b6a2b910db0542e762633d5a06a0b626068421d0e2...,663e226eadd6fa32dd9e529d2ff12db21c3b96df0ce9e0...,2019-11-05,STORE,383,24104,000000000000002.,0000000000000080.25,0,0,STORES,N,N
4,beb25901330a5ca04db94af2c273f352ac5747fd4c38c1...,180354f65d60f089e29a6b36e0f3138e2e67859fbcee90...,2019-11-07,STORE,4489,84043,000000000000000.,0000000000000000.00,0,0,NONE,N,N
5,5e2ccf97bef6bccef0b672b7a84f80400c8b0c9a9ff740...,3e00b2bf271180adf700164836d5be48fcb37588e1af09...,2019-11-08,STORE,1546,72901,000000000000001.,0000000000000027.00,0,0,STORES,N,N
6,aa7a82c3c0610cf10debdcb411d791a309ee468323b21a...,202f392c62fc1a92cb476188cb9714788897cc2a23d7c1...,2019-11-06,ONLINE,6990,NaN,000000000000000.,0000000000000000.00,0,0,NONE,N,N
7,0d16fa63853da8ab40311b57e2f935d2fdca7de16cf471...,8f37bfdc91804e700a75166fb5be0e8823900897735f56...,2019-11-07,STORE,5295,08840,000000000000001.,0000000000000148.87,0,0,STORES,N,N
8,6cc2587a2c9525ceb28fdb1d310893dc1d3dbdd476c036...,50be8e047928de08d6e17de160fe036a96199d8fdc2151...,2019-11-09,ONLINE,6990,NaN,000000000000000.,0000000000000000.00,0,0,NONE,N,N
9,b9f02ffdaef5edc66306571035ada69f1b6e82d2efbb83...,1ad8171e60653ed62279473b496a7c567358e9b006b143...,2019-11-03,STORE,45,43616,000000000000001.,0000000000000299.99,0,0,STORES,N,N


In [12]:
mapping_division_class=pd.read_table("/home/jian/BigLots/static_files/ProductTaxonomy/MediaStormProductTaxonomy20191101-134011-956.txt",
                                    dtype=str,sep="|",usecols=['division_id','class_code_id']).drop_duplicates()
print(mapping_division_class.shape)
mapping_division_name=pd.read_table("/home/jian/BigLots/static_files/MediaStorm Data Extract - Division Names.txt",sep="|",dtype=str)
mapping_division_class=pd.merge(mapping_division_class,mapping_division_name,on="division_id",how="left")
print(mapping_division_class.shape)

(463, 2)
(463, 3)


In [13]:
shopper_by_day_department_list=[]
shopper_by_day_list=[]
i_counter=0
for file in list_all_POS_item:
    df=pd.read_table(file,dtype=str,sep="|",usecols=['customer_id_hashed','transaction_dt','transaction_id','location_id','class_code_id','item_transaction_amt'])
    df=df[pd.notnull(df['customer_id_hashed'])]
    df=df[df['customer_id_hashed'].isin(list_ids_new_signs_since_Sep18)]
    if i_counter==0:
        df=df[df['transaction_dt']>=track_start_date]
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    #
    df_sales_depart=df.groupby(['customer_id_hashed','transaction_dt','class_code_id'])['item_transaction_amt'].sum().to_frame().reset_index().rename(columns={"item_transaction_amt":"sales"})
    shopper_by_day_department_list.append(df_sales_depart)
    
    #
    df_sales=df.groupby(['customer_id_hashed','transaction_dt'])['item_transaction_amt'].sum().to_frame().reset_index().rename(columns={"item_transaction_amt":"sales"})
    #
    df=df[df['item_transaction_amt']>0]

    df_trans=df[['customer_id_hashed','transaction_dt','transaction_id','location_id']].drop_duplicates()
    df_trans=df_trans.groupby(['customer_id_hashed','transaction_dt'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"trans"})
    
    df=pd.merge(df_sales,df_trans,on=['customer_id_hashed','transaction_dt'],how="outer")
    shopper_by_day_list.append(df)
    i_counter+=1
    if i_counter%10==1:
        print(i_counter,datetime.datetime.now())
df_shopper_by_month=pd.concat(shopper_by_day_list)
print(df_shopper_by_month.shape)

del shopper_by_day_list
df_shopper_by_month['shop_month']=df_shopper_by_month['transaction_dt'].apply(lambda x: x[:7])
df_shopper_by_month=df_shopper_by_month.groupby(['customer_id_hashed','shop_month'])['sales','trans'].sum().reset_index()

print(df_shopper_by_month.shape)

gc.collect()
print(datetime.datetime.now())


1 2019-11-18 19:05:25.000143
11 2019-11-18 19:12:51.058081
21 2019-11-18 19:24:12.891904
31 2019-11-18 19:32:30.706606
41 2019-11-18 19:41:20.493426
51 2019-11-18 19:50:28.416372
61 2019-11-18 19:59:30.013381
(15135992, 4)
(11054515, 4)
2019-11-18 20:02:49.725449


In [14]:
df_shopper_by_day_class=pd.concat(shopper_by_day_department_list)
df_shopper_by_day_class=pd.merge(df_shopper_by_day_class,mapping_division_class,on="class_code_id",how="left")
gc.collect()

print(datetime.datetime.now())

df_shopper_by_day_class.head(3)

2019-11-18 20:03:43.571823


,customer_id_hashed,transaction_dt,class_code_id,sales,division_id,division_desc
0,00002f57890442353de21f2ab1b17556089a69ad211c66...,2018-09-01,35039,20.0,4,SOFTHOME
1,00002f57890442353de21f2ab1b17556089a69ad211c66...,2018-09-01,35045,35.0,4,SOFTHOME
2,0003ca38cdd0e0b46b6971904e5ded3d313705e7b3cb7c...,2018-09-01,11001,5.0,1,FOOD


In [30]:
df_shopper_by_day_division=df_shopper_by_day_class.groupby(["customer_id_hashed","transaction_dt","division_id","division_desc"])['sales'].sum().to_frame().reset_index()
df_shopper_by_day_division.head(2)

NameError: name 'df_shopper_by_day_class' is not defined

In [16]:
del df_shopper_by_day_class
gc.collect()

0

# 1. Id count

In [17]:
df_shopper_by_month['shopping_behavior']=1

In [18]:
def month_add_interval(str_YYYY_mm,int_m_interval):
    YYYY_int=int(str_YYYY_mm.split("-")[0])
    mm_int=int(str_YYYY_mm.split("-")[1])
    
    mm_int_new=mm_int+int_m_interval
    if mm_int_new>=13:
        YYYY_int+=1
        mm_int_new=mm_int_new-12
    return str(YYYY_int)+"-"+str(mm_int_new).zfill(2)

In [19]:
df_output_1_count=df_output_long.copy()
for i in range(19):
    df_output_1_count["M"+str(i)]=df_output_1_count['registerd_month'].apply(lambda x: month_add_interval(x,i))
    df_output_1_count=pd.merge(df_output_1_count,df_shopper_by_month[['customer_id_hashed','shop_month','shopping_behavior']],
                            left_on=['customer_id_hashed',"M"+str(i)],
                            right_on=['customer_id_hashed',"shop_month"],how="left")
    del df_output_1_count['shop_month']
    del df_output_1_count["M"+str(i)]
    df_output_1_count['shopping_behavior']=df_output_1_count['shopping_behavior'].fillna(0)
    df_output_1_count=df_output_1_count.rename(columns={"shopping_behavior":"M"+str(i)})
    print(i,datetime.datetime.now())
    
print(df_output_1_count.shape)

0 2019-11-18 20:08:01.673899
1 2019-11-18 20:09:10.542151
2 2019-11-18 20:10:16.951910
3 2019-11-18 20:11:21.526948
4 2019-11-18 20:12:27.159289
5 2019-11-18 20:13:33.125472
6 2019-11-18 20:14:41.168308
7 2019-11-18 20:15:49.496636
8 2019-11-18 20:16:57.480555
9 2019-11-18 20:18:06.677358
10 2019-11-18 20:19:16.646192
11 2019-11-18 20:20:28.543514
12 2019-11-18 20:21:41.541537
13 2019-11-18 20:22:54.906488
14 2019-11-18 20:24:08.461834
15 2019-11-18 20:25:23.278162
16 2019-11-18 20:26:39.493209
17 2019-11-18 20:27:56.780376
18 2019-11-18 20:29:14.872433
(8221880, 22)


In [20]:
df_id_count=df_output_1_count.groupby("registerd_month")['customer_id_hashed'].count().to_frame().reset_index().rename(columns={"customer_id_hashed":"id_new_signed_count"})
df_id_shopper_count_sum=df_output_1_count.groupby(['registerd_month'])[[x for x in df_output_1_count.columns.tolist() if x[0]=="M"]].sum().reset_index()

df_output_1_count=pd.merge(df_id_count,df_id_shopper_count_sum,on="registerd_month",how="outer")

In [21]:
df_output_1_count

,registerd_month,id_new_signed_count,M0,M1,M2,M3,M4,M5,M6,M7,...,M9,M10,M11,M12,M13,M14,M15,M16,M17,M18
0,2018-09,559549,390823.0,90361.0,87480.0,89580.0,59316.0,53315.0,59465.0,57168.0,...,52844.0,54849.0,53022.0,54257.0,53007.0,21970.0,0.0,0.0,0.0,0.0
1,2018-10,441767,261677.0,73787.0,67800.0,42327.0,38221.0,41935.0,39953.0,38174.0,...,36938.0,35403.0,36248.0,38088.0,15608.0,0.0,0.0,0.0,0.0,0.0
2,2018-11,824107,466209.0,136546.0,74056.0,63221.0,69674.0,65457.0,62501.0,58977.0,...,57444.0,59262.0,62430.0,27366.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2018-12,843470,484238.0,78858.0,60452.0,64319.0,60109.0,56290.0,54078.0,56224.0,...,51853.0,54453.0,22529.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-01,554172,357651.0,79372.0,71564.0,62632.0,57048.0,53218.0,54413.0,50388.0,...,49653.0,20623.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2019-02,495334,317587.0,78865.0,57211.0,50433.0,45844.0,46079.0,43145.0,41748.0,...,17433.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2019-03,551970,373413.0,84743.0,62312.0,54405.0,53883.0,49178.0,47412.0,47451.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2019-04,529464,352694.0,78352.0,59453.0,56434.0,49932.0,48044.0,47322.0,19131.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,2019-05,518120,355840.0,77480.0,61986.0,53535.0,49193.0,48066.0,19385.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2019-06,515025,360205.0,85111.0,60986.0,53499.0,51413.0,20488.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 2. Sales

In [22]:
df_output_2_sales=df_output_long.copy()
for i in range(19):
    df_output_2_sales["M"+str(i)]=df_output_2_sales['registerd_month'].apply(lambda x: month_add_interval(x,i))
    df_output_2_sales=pd.merge(df_output_2_sales,df_shopper_by_month[['customer_id_hashed','shop_month','sales']],
                            left_on=['customer_id_hashed',"M"+str(i)],
                            right_on=['customer_id_hashed',"shop_month"],how="left")
    del df_output_2_sales['shop_month']
    del df_output_2_sales["M"+str(i)]
    df_output_2_sales['sales']=df_output_2_sales['sales'].fillna(0)
    df_output_2_sales=df_output_2_sales.rename(columns={"sales":"M"+str(i)})
    print(i,datetime.datetime.now())
print(df_output_2_sales.shape)


0 2019-11-18 20:30:26.186556
1 2019-11-18 20:31:31.181606
2 2019-11-18 20:32:42.378487
3 2019-11-18 20:33:48.291760
4 2019-11-18 20:34:57.504310
5 2019-11-18 20:36:05.689577
6 2019-11-18 20:37:19.077166
7 2019-11-18 20:38:38.350363
8 2019-11-18 20:39:55.944449
9 2019-11-18 20:41:16.242282
10 2019-11-18 20:42:35.955539
11 2019-11-18 20:43:59.228352
12 2019-11-18 20:45:14.434720
13 2019-11-18 20:46:37.573479
14 2019-11-18 20:48:01.530703
15 2019-11-18 20:49:20.041119
16 2019-11-18 20:50:39.025668
17 2019-11-18 20:52:08.649830
18 2019-11-18 20:53:39.034989
(8221880, 22)


In [23]:
df_id_shopper_sales_sum=df_output_2_sales.groupby(['registerd_month'])[[x for x in df_output_2_sales.columns.tolist() if x[0]=="M"]].sum().reset_index()

df_output_2_sales=pd.merge(df_id_count,df_id_shopper_sales_sum,on="registerd_month",how="outer")

# 3. Trans

In [24]:
df_output_3_trans=df_output_long.copy()
for i in range(19):
    df_output_3_trans["M"+str(i)]=df_output_3_trans['registerd_month'].apply(lambda x: month_add_interval(x,i))
    df_output_3_trans=pd.merge(df_output_3_trans,df_shopper_by_month[['customer_id_hashed','shop_month','trans']],
                            left_on=['customer_id_hashed',"M"+str(i)],
                            right_on=['customer_id_hashed',"shop_month"],how="left")
    del df_output_3_trans['shop_month']
    del df_output_3_trans["M"+str(i)]
    df_output_3_trans['trans']=df_output_3_trans['trans'].fillna(0)
    df_output_3_trans=df_output_3_trans.rename(columns={"trans":"M"+str(i)})
    print(i,datetime.datetime.now())
print(df_output_3_trans.shape)


0 2019-11-18 20:54:54.890706
1 2019-11-18 20:56:00.068404
2 2019-11-18 20:57:07.976380
3 2019-11-18 20:58:18.643705
4 2019-11-18 20:59:33.102319
5 2019-11-18 21:00:40.872497
6 2019-11-18 21:01:55.261469
7 2019-11-18 21:03:13.961339
8 2019-11-18 21:04:31.124603
9 2019-11-18 21:05:46.907401
10 2019-11-18 21:07:02.115547
11 2019-11-18 21:08:25.183633
12 2019-11-18 21:09:48.427046
13 2019-11-18 21:11:13.244360
14 2019-11-18 21:12:38.554772
15 2019-11-18 21:14:06.444105
16 2019-11-18 21:15:34.549650
17 2019-11-18 21:17:04.551446
18 2019-11-18 21:18:38.045839
(8221880, 22)


In [25]:
df_id_shopper_trans_sum=df_output_3_trans.groupby(['registerd_month'])[[x for x in df_output_3_trans.columns.tolist() if x[0]=="M"]].sum().reset_index()

df_output_3_trans=pd.merge(df_id_count,df_id_shopper_trans_sum,on="registerd_month",how="outer")

In [26]:
writer=pd.ExcelWriter("./BL_LTV_retention_rate_for_new_sign_ups_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_output_1_count.to_excel(writer,"table_1_count",index=False)
df_output_2_sales.to_excel(writer,"table_2_sales",index=False)
df_output_3_trans.to_excel(writer,"table_3_trans",index=False)

writer.save()

# Sales by Division table

In [36]:
df_shopper_by_day_division['transaction_dt']=pd.to_datetime(df_shopper_by_day_division['transaction_dt'],format="%Y-%m-%d").dt.date
df_output_long['sign_up_date']=pd.to_datetime(df_output_long['sign_up_date'],format="%Y-%m-%d").dt.date
df_output_long=pd.merge(df_output_long,df_shopper_by_day_division,on="customer_id_hashed",how="left")



In [57]:
df_output_long['day_diff']=(df_output_long['transaction_dt']-df_output_long['sign_up_date']).dt.days
df_output_long['month_diff']=np.floor(df_output_long['day_diff']/30)

df_output_long.head(2)

,customer_id_hashed,sign_up_date,registerd_month,transaction_dt,division_id,division_desc,sales,day_diff,month_diff,month_diff_str
0,b427a86927696538066305dd91ae960734488531b44d81...,2019-11-09,2019-11,2019-11-09,5,HARDHOME,49.99,0.0,0.0,M00
1,4f78a082263f3bf1e27759770ba7199774b71f3078046d...,2019-11-09,2019-11,NaN,NaN,NaN,NaN,NaN,NaN,Mnan


In [58]:
df_output_long[df_output_long['month_diff']<0].shape

(51326, 10)

In [59]:
df_output_long.shape

(34652207, 10)

In [60]:
df_output_long[df_output_long['month_diff']<0]['customer_id_hashed'].nunique()

20634

In [62]:
df_output_long['customer_id_hashed'].nunique()

8221880

In [64]:
20634/8221880

0.00250964499603497

In [65]:
df_output_long['month_diff_str']=df_output_long['month_diff'].apply(lambda x: "M_"+str(x).split(".")[0].zfill(2))

In [69]:
df_output_pivot_0=df_output_long.groupby(['registerd_month','division_id','division_desc','month_diff_str'])['sales'].sum().to_frame().reset_index()
df_output_pivot_0=df_output_pivot_0.pivot_table(index=['registerd_month','division_id','division_desc'],columns="month_diff_str",values="sales").reset_index()

In [73]:
df_output_pivot_0['division_id']=df_output_pivot_0['division_id'].astype(int)
df_output_pivot_0=df_output_pivot_0.sort_values(["registerd_month",'division_id'])

In [75]:
df_output_pivot_1['division_desc'].unique()

array(['FOOD', 'CONSUMABLES', 'SEASONAL', 'SOFTHOME', 'HARDHOME',
       'ELECNACCES', 'DIVISION 8 (EMP DISCOUNT)', 'FURNITURE',
       'DIVISION 10'], dtype=object)

In [76]:
df_output_pivot_1=df_output_pivot_0[[x for x  in df_output_pivot_0.columns.tolist() if "-" not in x]]
df_output_pivot_2=df_output_pivot_1[df_output_pivot_1['division_desc']!="DIVISION 8 (EMP DISCOUNT)"]
df_output_pivot_2=df_output_pivot_2[df_output_pivot_2['division_desc']!="DIVISION 10"]


In [77]:
writer=pd.ExcelWriter("./BL_LTV_by_division_rolling_months_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_output_pivot_2.to_excel(writer,"table_final",index=False)
df_output_pivot_1.to_excel(writer,"table_exclude_outlier",index=False)
df_output_pivot_0.to_excel(writer,"table_original",index=False)
writer.save()

In [78]:
df_output_pivot_2

month_diff_str,registerd_month,division_id,division_desc,M_00,M_01,M_02,M_03,M_04,M_05,M_06,M_07,M_08,M_09,M_10,M_11,M_12,M_13,M_14
0,2018-09,1,FOOD,2.814830e+06,734282.82,816516.23,628402.74,502339.38,489749.20,510692.36,458800.25,428792.43,448575.08,426329.92,442229.95,468820.27,405940.87,59458.52
2,2018-09,2,CONSUMABLES,3.320054e+06,727022.44,788554.75,664813.50,538203.88,548558.19,554860.28,508824.52,519795.94,551637.02,542279.37,533644.03,539834.87,432881.64,66925.25
3,2018-09,3,SEASONAL,2.323820e+06,710964.16,1402825.31,485726.30,196300.46,318131.28,638010.65,724381.63,730614.75,578992.47,386795.41,260395.18,282826.38,289622.06,51509.98
4,2018-09,4,SOFTHOME,6.444454e+06,932634.51,961422.84,742285.79,593285.47,667715.09,655392.94,548139.82,505109.85,513523.94,533821.84,567231.13,597088.58,474437.91,71837.54
5,2018-09,5,HARDHOME,2.326243e+06,420366.66,504203.96,346237.97,221719.24,230970.63,229070.09,195222.02,182639.60,222163.81,215890.48,198932.27,203426.89,177710.06,26579.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,2019-11,3,SEASONAL,1.077182e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130,2019-11,4,SOFTHOME,1.497465e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131,2019-11,5,HARDHOME,5.460396e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132,2019-11,6,ELECNACCES,4.872772e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
